In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
response = requests.get('https://www.thecocktaildb.com/api/json/v1/1/search.php?s=margarita')

In [3]:
response

<Response [200]>

In [4]:
#page = BeautifulSoup(response.text, 'html5')

In [5]:
page = response.json()

In [6]:
#name of drink
page['drinks'][0]['strDrink']

'Margarita'

In [7]:
#found drink id
drink_id = page['drinks'][0]['idDrink']
drink_id

'11007'

# Lots of information on generic pull
testing if id specific pull has better information

In [8]:
response = requests.get('https://www.thecocktaildb.com/api/json/v1/1/lookup.php?i={}'.format(drink_id))

page = response.json()
page

# Unfortunately it is exactly the same
at least I do not need to scrape twice once for ids and once for ingredients

In [9]:
response = requests.get('https://www.thecocktaildb.com/api/json/v1/1/search.php?s=irish_coffee')
page = response.json()
page['drinks'][0]['strIngredient1']

'Irish whiskey'

In [10]:
page['drinks'][0]['strDrink']

'Irish Coffee'

In [11]:
page['drinks'][0]

{'idDrink': '13971',
 'strDrink': 'Irish Coffee',
 'strDrinkAlternate': None,
 'strTags': 'IBA,ContemporaryClassic',
 'strVideo': None,
 'strCategory': 'Coffee / Tea',
 'strIBA': 'Contemporary Classics',
 'strAlcoholic': 'Alcoholic',
 'strGlass': 'Irish coffee cup',
 'strInstructions': 'Heat the coffee, whiskey and sugar; do not boil. Pour into glass and top with cream; serve hot.',
 'strInstructionsES': None,
 'strInstructionsDE': 'Kaffee, Whiskey und Zucker erhitzen; nicht kochen lassen. In ein Glas geben und mit Sahne übergießen; heiß servieren.',
 'strInstructionsFR': None,
 'strInstructionsIT': 'Riscaldare il caffè, il whisky e lo zucchero; non bollire. Versare nel bicchiere e ricoprire con la panna; servito caldo.',
 'strInstructionsZH-HANS': None,
 'strInstructionsZH-HANT': None,
 'strDrinkThumb': 'https://www.thecocktaildb.com/images/media/drink/sywsqw1439906999.jpg',
 'strIngredient1': 'Irish whiskey',
 'strIngredient2': 'Coffee',
 'strIngredient3': 'Sugar',
 'strIngredient4':

In [12]:
#each pull has 15 ingredient slots, starting at 1

ngredients = []
for i in range(1,16):
    item = page['drinks'][0]['strIngredient{}'.format(i)]
    if item is not None:
        ingredients.append(item)
        

In [13]:
ingredients

['Irish whiskey', 'Coffee', 'Sugar', 'Whipped cream']

In [14]:
# even instructions how to make them!
instructions = page['drinks'][0]['strInstructions']
instructions

'Heat the coffee, whiskey and sugar; do not boil. Pour into glass and top with cream; serve hot.'

In [22]:
#list of all cocktails I'm starting with
cocktail_list = ['cosmopolitan', 'margarita', 'mojito', 'old_fashioned', 'pina_colada',
                'irish_coffee', 'moscow_mule', 'mimosa', 'aperol_spritz', 'bloody_mary']

cocktail_dict ={}

for items in cocktail_list:
    response = requests.get('https://www.thecocktaildb.com/api/json/v1/1/search.php?s={}'.format(items))
    page = response.json()
    
#     if items != page['drinks'][0]['strDrink']:
#         raise ValueError('Cocktals do not Match')

    #including cocktail name to save time on reseting index and reformatting the names to match the pictures
    cocktail = page['drinks'][0]['strDrink']
    
    ingredients = []
    for i in range(1,16):
        item = page['drinks'][0]['strIngredient{}'.format(i)]
        if item is not None:
            ingredients.append(item)
            
    instructions = page['drinks'][0]['strInstructions']
    
    cocktail_dict[items] = (cocktail, ingredients, instructions)
        
    

In [23]:
cocktail_dict

{'cosmopolitan': ('Cosmopolitan',
  ['Absolut Citron', 'Lime juice', 'Cointreau', 'Cranberry juice'],
  'Add all ingredients into cocktail shaker filled with ice. Shake well and double strain into large cocktail glass. Garnish with lime wheel.'),
 'margarita': ('Margarita',
  ['Tequila', 'Triple sec', 'Lime juice', 'Salt'],
  'Rub the rim of the glass with the lime slice to make the salt stick to it. Take care to moisten only the outer rim and sprinkle the salt on it. The salt should present to the lips of the imbiber and never mix into the cocktail. Shake the other ingredients with ice, then carefully pour into the glass.'),
 'mojito': ('Mojito',
  ['Light rum', 'Lime', 'Sugar', 'Mint', 'Soda water'],
  'Muddle mint leaves with sugar and lime juice. Add a splash of soda water and fill the glass with cracked ice. Pour the rum and top with soda water. Garnish and serve with straw.'),
 'old_fashioned': ('Old Fashioned',
  ['Bourbon', 'Angostura bitters', 'Sugar', 'Water'],
  'Place sugar

In [17]:
cocktail_dict['cosmopolitan']

(['Absolut Citron', 'Lime juice', 'Cointreau', 'Cranberry juice'],
 'Add all ingredients into cocktail shaker filled with ice. Shake well and double strain into large cocktail glass. Garnish with lime wheel.')

In [24]:
df = pd.DataFrame(cocktail_dict).T

In [29]:
df.columns = ['cocktail', 'ingredients', 'recipie']

In [27]:
df

,coctail,ingredients,recipie
cosmopolitan,Cosmopolitan,"[Absolut Citron, Lime juice, Cointreau, Cranbe...",Add all ingredients into cocktail shaker fille...
margarita,Margarita,"[Tequila, Triple sec, Lime juice, Salt]",Rub the rim of the glass with the lime slice t...
mojito,Mojito,"[Light rum, Lime, Sugar, Mint, Soda water]",Muddle mint leaves with sugar and lime juice. ...
old_fashioned,Old Fashioned,"[Bourbon, Angostura bitters, Sugar, Water]",Place sugar cube in old fashioned glass and sa...
pina_colada,Pina Colada,"[Light rum, Coconut milk, Pineapple]",Mix with crushed ice in blender until smooth. ...
irish_coffee,Irish Coffee,"[Irish whiskey, Coffee, Sugar, Whipped cream]","Heat the coffee, whiskey and sugar; do not boi..."
moscow_mule,Moscow Mule,"[Vodka, Lime juice, Ginger ale]",Combine vodka and ginger beer in a highball gl...
mimosa,Mimosa,"[Champagne, Orange juice]","Ensure both ingredients are well chilled, then..."
aperol_spritz,Aperol Spritz,"[Aperol, Prosecco, Soda Water, , , , ]",Put a couple of cubes of ice into 2 glasses an...
bloody_mary,Bloody Mary,"[Vodka, Tomato juice, Lemon juice, Worcestersh...","Stirring gently, pour all ingredients into hig..."


In [30]:
# saving dataframe to transfer to another notebook
df.to_csv(r'/users/michaelharnett/desktop/metis/projects/cocktail_classifier/data/recipie_df.csv', index=False)